Step 1: Import Python Libraries into the Jupyter Notebook.

Note: Please download the relevant Python packages to execute this notebook.

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import html2text
import urllib.request
from user_agent import generate_user_agent
from sklearn.preprocessing import StandardScaler

Step 2: Define a function get_molecular_descriptors to web-scrap the relevant c-MolDes data for a given drug entry (represented by DB_ID) in the DrugBank database. If the c-MolDes properties for a given drug (represented by the DB_ID) are not be published in the DrugBank database, then the data for that entry are zero padded.

In [4]:
def get_molecular_descriptors(content, verbose):
    # Molecular Descriptors
    if verbose==0:
        print("Fifteen Molecular Descriptors:\n")
        
    test=content.split("Predicted Properties")[1]
    molecular_descriptors=["<td>logP</td><td>", "<td>logS</td><td>", "<td>Physiological Charge</td>", 
                           "<td>Hydrogen Acceptor Count</td>", "<td>Hydrogen Donor Count</td>", 
                           "<td>Polar Surface Area</td>", "<td>Rotatable Bond Count</td>", 
                           "<td>Polarizability</td>", "<td>Number of Rings</td>", "<td>Refractivity</td><td>",
                          "<td>Bioavailability</td><td>", "<td>Rule of Five</td><td>", "<td>Ghose Filter</td><td>",
                          "<td>Veber's Rule</td><td>", "<td>MDDR-like Rule</td><td>", "<td>Water Solubility</td><td>"]
    #0-logP
    fragment_logP=test.split(molecular_descriptors[0])
    if len(fragment_logP)>1:
        if "ALOGPS" in fragment_logP[1]:
            #ALOGPS
            logP=fragment_logP[1].split("</td>")[0]
        else:
            logP=0
    else:
        logP=0
        if verbose==0:
            print("Did not find logP ALOGPS at fragment[1]")

    #1-logS
    fragment_logS=test.split(molecular_descriptors[1])
    if len(fragment_logS)>1:
        if "ALOGPS" in fragment_logS[1]:
            #ALOGPS
            logS=fragment_logS[1].split("</td>")[0]
        else:
            logS=0
    else:
        logS=0
        if verbose==0:
            print("Did not find logS ALOGPS at fragment[1]")


    #2-Physiological Charge
    fragment_PhysCharge=test.split(molecular_descriptors[2])
    if "ChemAxon" in fragment_PhysCharge[1]:
        #ChemAxon
        PhysCharge=fragment_PhysCharge[1].split("<td>")[1].split("</td")[0]
    else:
        PhysCharge=0
        if verbose==0:
            print("Did not find Physiological Charge ChemAxon at fragment[1]")

    #3-Hydrogen Acceptor Count
    fragment_HAccept=test.split(molecular_descriptors[3])
    if "ChemAxon" in fragment_HAccept[1]:
        #ChemAxon
        HAccept=fragment_HAccept[1].split("<td>")[1].split("</td")[0]
    else:
        HAccept=0
        if verbose==0:
            print("Did not find Hydrogen Acceptor Count ChemAxon at fragment[1]")

    #4-Hydrogen Donor Count
    fragment_HDonor=test.split(molecular_descriptors[4])
    if "ChemAxon" in fragment_HDonor[1]:
        #ChemAxon
        HDonor=fragment_HDonor[1].split("<td>")[1].split("</td")[0]
    else:
        HDonor=0
        if verbose==0:
            print("Did not find Hydrogen Donor Count ChemAxon at fragment[1]")

    #5-Polar Surface Area in (Å2) unless otherwise stated
    fragment_PolSA=test.split(molecular_descriptors[5])
    if "ChemAxon" in fragment_PolSA[1]:
        #ChemAxon
        PolSA=fragment_PolSA[1].split("<td>")[1].split("</td")[0].split("Å<sup>2</sup>")[0]
    else:
        PolSA=0
        if verbose==0:
            print("Did not find Polar Surface Area in Å<sup>2</sup> ChemAxon at fragment[1]")

    #6-Rotatable Bond Count
    fragment_RotaBondCount=test.split(molecular_descriptors[6])
    if "ChemAxon" in fragment_RotaBondCount[1]:
        #ChemAxon
        RotaBondCount=fragment_RotaBondCount[1].split("<td>")[1].split("</td")[0]
    else:
        RotaBondCount=0
        if verbose==0:
            print("Did not find Rotatable Bond Count ChemAxon at fragment[1]")

    #7-Polarizability in Å<sup>3</sup> unless otherwise stated
    fragment_Polarizability=test.split(molecular_descriptors[7])
    if "ChemAxon" in fragment_Polarizability[1]:
        #ChemAxon
        Polarizability=fragment_Polarizability[1].split("<td>")[1].split("</td")[0].split("Å<sup>3</sup>")[0]
    else:
        Polarizability=0
        if verbose==0:
            print("Did not find Polarizability ChemAxon at fragment[1]")

    #8-Number of Rings
    fragment_NRings=test.split(molecular_descriptors[8])
    if "ChemAxon" in fragment_NRings[1]:
        #ChemAxon
        NRings=fragment_NRings[1].split("<td>")[1].split("</td")[0]
    else:
        NRings=0
        if verbose==0:
            print("Did not find Number of Rings ChemAxon at fragment[1]")
            
    #9-Refractivity in m3·mol-1 unless otherwise stated
    fragment_Refractivity=test.split(molecular_descriptors[9])
    if len(fragment_Refractivity)>1:
        if "ChemAxon" in fragment_Refractivity[1]:
            #ALOGPS
            f2_Refractivity=fragment_Refractivity[1].split("</td>")[0]
            if "m<sup>3</sup>·mol<sup>-1</sup>" in f2_Refractivity:
                Refractivity=f2_Refractivity.split(" ")[0]
        else:
            Refractivity="NA"
    else:
        Refractivity="NA"
        if verbose==0:
            print("Did not find Refractivity at fragment[1]")
            
    #10-Bioavailability
    fragment_Bioavail=test.split(molecular_descriptors[10])
    if len(fragment_Bioavail)>1:
        if "ChemAxon" in fragment_Bioavail[1]:
            #ALOGPS
            Bioavail=fragment_Bioavail[1].split("</td>")[0]
        else:
            Bioavail="NA"
    else:
        Bioavail="NA"
        if verbose==0:
            print("Did not find Bioavailability at fragment[1]")
            
    #11-Rule of Five
    fragment_RoF=test.split(molecular_descriptors[11])
    if len(fragment_RoF)>1:
        if "ChemAxon" in fragment_RoF[1]:
            #ALOGPS
            RoF_stat=fragment_RoF[1].split("</td>")[0]
            if RoF_stat=="Yes":
                RoF=1
            elif RoF_stat=="No":
                RoF=0
        else:
            RoF="NA"
    else:
        RoF="NA"
        if verbose==0:
            print("Did not find Rule of Five at fragment[1]")
            
    #12-Ghose Filter
    fragment_GF=test.split(molecular_descriptors[12])
    if len(fragment_GF)>1:
        if "ChemAxon" in fragment_GF[1]:
            #ALOGPS
            GF_stat=fragment_GF[1].split("</td>")[0]
            if GF_stat=="Yes":
                GF=1
            elif GF_stat=="No":
                GF=0
        else:
            GF="NA"
    else:
        GF="NA"
        if verbose==0:
            print("Did not find Ghose Filter at fragment[1]")
            
    #13-Veber's Rule
    fragment_VR=test.split(molecular_descriptors[13])
    if len(fragment_VR)>1:
        if "ChemAxon" in fragment_VR[1]:
            #ALOGPS
            VR_stat=fragment_VR[1].split("</td>")[0]
            if VR_stat=="Yes":
                VR=1
            elif VR_stat=="No":
                VR=0
        else:
            VR="NA"
    else:
        VR="NA"
        if verbose==0:
            print("Did not find Veber's Rule at fragment[1]")
            
    #14-MDDR-like Rule
    fragment_MDDR=test.split(molecular_descriptors[14])
    if len(fragment_MDDR)>1:
        if "ChemAxon" in fragment_MDDR[1]:
            #ALOGPS
            MDDR_stat=fragment_MDDR[1].split("</td>")[0]
            if MDDR_stat=="Yes":
                MDDR=1
            elif MDDR_stat=="No":
                MDDR=0
        else:
            MDDR="NA"
    else:
        MDDR="NA"
        if verbose==0:
            print("Did not find MDDR-like Rule at fragment[1]")
            
    #15-Water Solubility
    fragment_WS=test.split(molecular_descriptors[15])
    if len(fragment_WS)>1:
        if "ALOGPS" in fragment_WS[1]:
            #ALOGPS
            f2_WS=fragment_WS[1].split("</td>")[0]
            if "mg/mL" in f2_WS:
                WS=f2_WS.split(" ")[0]
        else:
            WS="NA"
    else:
        WS="NA"
        if verbose==0:
            print("Did not find Water Solubility at fragment[1]")
            
    if verbose==0:
        print('{0:25}'.format('0) logP'), "=", logP)
        print('{0:25}'.format('1) logS'), "=", logS)
        print('{0:25}'.format('2) Physiological Charge'), "=", PhysCharge)
        print('{0:25}'.format('3) H Acceptor Count'), "=", HAccept)
        print('{0:25}'.format('4) H Donor Count'), "=", HDonor)
        print('{0:25}'.format('5) Polar Surface Area (Å2)'), "=", PolSA)
        print('{0:25}'.format('6) Rotatable Bond Count'), "=", RotaBondCount)
        print('{0:25}'.format('7) Polarizability (Å3)'), "=", Polarizability)
        print('{0:25}'.format('8) Number of Rings'), "=", NRings)
        print('{0:25}'.format('9) Refractivity'), "=", Refractivity)
        print('{0:25}'.format('10) Bioavailability'), "=", Bioavail)
        print('{0:25}'.format('11) Rule of Five'), "=", RoF)
        print('{0:25}'.format('12) Ghose Filter'), "=", GF)
        print('{0:25}'.format('13) Veber\'s Rule'), "=", VR)
        print('{0:25}'.format('14) MDDR-like Rule'), "=", MDDR)
        print('{0:25}'.format('15) Water Solubility'), "=", WS)
        
    result_molecular_descriptors=[logP, logS, PhysCharge, HAccept, HDonor, 
                                  PolSA, RotaBondCount, Polarizability, 
                                  NRings, Refractivity, Bioavail, RoF, GF, VR, MDDR, WS]
    
    return result_molecular_descriptors

def na_molecular_descriptors_default_protocol():
    #For ['DB12598', 'DB06219', 'DB00375']
    logP=0
    logS=0
    PhysCharge=0
    HAccept=0
    HDonor=0
    PolSA=0
    RotaBondCount=0
    Polarizability=0
    NRings=0
    Refractivity=0
    Bioavail=0
    RoF=0
    GF=0
    VR=0
    MDDR=0
    WS=0

    result_molecular_descriptors=[logP, logS, PhysCharge, HAccept, HDonor, 
                                  PolSA, RotaBondCount, Polarizability, 
                                  NRings, Refractivity, Bioavail, RoF, GF, VR, MDDR, WS]
    
    return result_molecular_descriptors

Step 3: Doing web-scrapping on DrugBank database on all 1,710 drugs used to develop the DDI MLP (>192,000 DDIs).

In [5]:
prefix_target_url="https://www.drugbank.ca/drugs/"

headers={}
headers["User-Agent"]=generate_user_agent()

DB_ID_SMILES_df=pd.read_csv("1_1_1_Input\DB_ID_SMILES.csv")
x=DB_ID_SMILES_df.loc[:, "DB_ID"].values

# Molecular Descriptors
Dict_pKA_SA={}
verbose=1
for i in range(len(x)):
    if i%500==0:
        print("Now at ", i)
        
    Drug=str(x[i])
    try:
        url=prefix_target_url+Drug

        resource = urllib.request.urlopen(url)
        content = resource.read()
        charset = resource.headers.get_content_charset()
        content = content.decode(charset)

        test=content.split("Predicted Properties")[1]
        molecular_descriptors=["<td>pKa (Strongest Acidic)</td><td>"]
        #9) pKa (Strongest Acidic)
        fragment_pKa_SA=test.split(molecular_descriptors[0])
        if len(fragment_pKa_SA)>1:
            if "ChemAxon" in fragment_pKa_SA[1]:
                #ALOGPS
                pKa_SA=fragment_pKa_SA[1].split("</td>")[0]
                Dict_pKA_SA[Drug]=pKa_SA
            else:
                Dict_pKA_SA[Drug]=[]
        else:
            Dict_pKA_SA[Drug]=[]
            if verbose==0:
                print("Did not find pKa (Strongest Acidic) at fragment[1] ", Drug)

    except:
        Dict_pKA_SA[Drug]=[]
        print("Cannot Find Url for ", Drug, "\n")
        
Dict_pKA_SB={}
for i in range(len(x)):
    if i%500==0:
        print("Now at ", i)
        
    Drug=str(x[i])
    try:
        url=prefix_target_url+Drug

        resource = urllib.request.urlopen(url)
        content = resource.read()
        charset = resource.headers.get_content_charset()
        content = content.decode(charset)

        test=content.split("Predicted Properties")[1]
        molecular_descriptors=["<td>pKa (Strongest Basic)</td><td>"]
        #10) pKa (Strongest Basic)
        fragment_pKa_SB=test.split(molecular_descriptors[0])
        if len(fragment_pKa_SB)>1:
            if "ChemAxon" in fragment_pKa_SB[1]:
                #ALOGPS
                pKa_SB=fragment_pKa_SB[1].split("</td>")[0]
                Dict_pKA_SB[Drug]=pKa_SB
            else:
                Dict_pKA_SB[Drug]=[]
        else:
            Dict_pKA_SB[Drug]=[]
            if verbose==0:
                print("Did not find pKa (Strongest Basic) at fragment[1] ", Drug)

    except:
        Dict_pKA_SB[Drug]=[]
        print("Cannot Find Url for ", Drug, "\n")
        
Dict_pKA={}
count=0
for key, val in Dict_pKA_SA.items():
    if Dict_pKA_SA[key]==[] and Dict_pKA_SB[key]==[]:
        Dict_pKA[key]=[str(0), str(0)]
    
    elif Dict_pKA_SA[key]==[] and Dict_pKA_SB[key]!=[]:
        Dict_pKA[key]=[Dict_pKA_SB[key], Dict_pKA_SB[key]]
        
    elif Dict_pKA_SA[key]!=[] and Dict_pKA_SB[key]==[]:
        Dict_pKA[key]=[Dict_pKA_SA[key], Dict_pKA_SA[key]]
        
    elif Dict_pKA_SA[key]!=[] and Dict_pKA_SB[key]!=[]:
        Dict_pKA[key]=[Dict_pKA_SA[key], Dict_pKA_SB[key]]
        
Dict_DB_Molecular_Descriptors={}
for i in range(len(x)):
    if i%500==0:
        print("Now at ", i)
        
    Drug=str(x[i])
    try:
        url=prefix_target_url+Drug

        resource = urllib.request.urlopen(url)
        content = resource.read()
        charset = resource.headers.get_content_charset()
        content = content.decode(charset)

        try:
            Dict_DB_Molecular_Descriptors[Drug]=get_molecular_descriptors(content, 1)
        except:
            Dict_DB_Molecular_Descriptors[Drug]=[]
            print("Encountered Problems in Extracting Molecular Descriptors for ", Drug, "\n")
    except:
        Dict_DB_Molecular_Descriptors[Drug]=[]
        print("Cannot Find Url for ", Drug, "\n")
        
unusual_format_DB=[]
for key, val in Dict_DB_Molecular_Descriptors.items():
    if val==[]:
        unusual_format_DB.append(key)
        
for i in range(len(unusual_format_DB)):
    Dict_DB_Molecular_Descriptors[unusual_format_DB[i]]=na_molecular_descriptors_default_protocol()
    
#Concatenate the pKA info
Dict_DB_Molecular_Descriptors_and_pKA={}

for key, val in Dict_DB_Molecular_Descriptors.items():
    Dict_DB_Molecular_Descriptors_and_pKA[key]=Dict_DB_Molecular_Descriptors[key]+Dict_pKA[key]
    if Dict_DB_Molecular_Descriptors_and_pKA[key][-3]=="NA":
        Dict_DB_Molecular_Descriptors_and_pKA[key][-3]=0


Done  0


Step 4: Export the raw c-ADMET Properties as "Raw_DDI_MLP_c-MolDes.csv"

In [ ]:
f = open('Raw_DDI_MLP_c-MolDes.csv', 'w+')
count=0
for key, val in Dict_DB_Molecular_Descriptors_and_pKA.items():
    if count==0:
        Line="DB_ID, logP, logS, PhysiologicalCharge, HydrogenAcceptor, HydrogenDonor, PolarSurfaceArea(A2), RotatableBondCount, Polarizability(A3), NumberofRings, Refractivity(m3.mol-1),Bioavailability, Rule of Five, Ghose Filter, Veber's Rule, MDDR-like Rule, Water Solubility(mg/mL), pKa(StrongestAcidic), pKa(StrongestBasic)\n"
        f.write(Line)  
        
    Line=str(key)+","+str(val[0])+","+str(val[1])+","+str(val[2])+","+str(val[3])+","+str(val[4])+","+str(val[5])+","+str(val[6])+","+str(val[7])+","+str(val[8])+","+str(val[9])+","+str(val[10])+","+str(val[11])+","+str(val[12])+","+str(val[13])+","+str(val[14])+","+str(val[15])+","+str(val[16])+","+str(val[17])+"\n"
    f.write(Line)  
    count=1

f.close()

Step 5: Standardize the raw c-MolDes data to accelerate the machine learning process later on.

In [2]:
MolDes_df=pd.read_csv("1_1_2_Output\Raw_DDI_MLP_c-MolDes.csv")
MolDes_Prop_Columns=MolDes_df.columns[1:]

MolDes_Prop_Values=MolDes_df.loc[:, MolDes_Prop_Columns].values
Standardized_MolDes_Prop_Values=StandardScaler().fit_transform(MolDes_Prop_Values)

new_MolDes_df = MolDes_df.filter([MolDes_df.columns[0]], axis=1)
MolDes_Prop_df=pd.DataFrame(Standardized_MolDes_Prop_Values, columns=MolDes_Prop_Columns)
Standardized_MolDes_df=pd.concat([new_MolDes_df, MolDes_Prop_df], axis=1)
Standardized_MolDes_df.to_csv(r'1_1_2_Output\DDI_MLP_Standardized_c-MolDes.csv', index=False)